In [106]:
path = "./defi_emb.txt"
from transformers import BertModel,BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/home/lawson/pretrain/bert-base-cased")
model = BertModel.from_pretrained("/home/lawson/pretrain/bert-base-cased")

In [86]:
'''简单的方式读取文件
'''
def simpleReadXml(pathData,pathLabel):
    import  xml.dom.minidom as dom
    # step1.先从pathLabel 中找出所有是双关语的句子
    allHomo = [] # 标记所有是双关语的句子集合 [hom_1,hom_2]
    allLabel =[] # 找出所有的双关词 [hom_1_12]
    punWord = [] # 表示所有的双关词
    with open(pathLabel,'r') as f:
        line = f.readline()
        while (line!=""):
            line = line.strip() # 去行末换行
            line = line.split() # 空格分割            
            #print(line[0])
            allHomo.append(line[0])
            allLabel.append(line[1]) # 将双关词的id放入其中
            line = f.readline()
    #print(allHomo)
    
    # step2.接着读取双关句，成为一行文本
    #打开xml文档
    dom2 = dom.parse(pathData)
    #得到文档元素对象
    root = dom2.documentElement    
    texts = root.getElementsByTagName("text") # 得到所有的text             
    puns = [] # 存储双关语的列表
    for text in texts:
        name = text.getAttribute('id')
        if name in allHomo:
            words = text.getElementsByTagName("word") #得到word
            pun = []
            for word in words:
                a = word.firstChild.data                
                pun.append(a)
                if word.getAttribute('id') in allLabel:
                    punWord.append(a)  # 这个单词就是双关词
            puns.append(pun)
    
    return puns,punWord  # 1607条语义双关语 以及 对应的双关词

In [87]:
# 最后运行的模块
pathData = "/home/lawson/program/data/puns/test/homo/subtask2-homographic-test.xml"
pathLabel = "/home/lawson/program/data/puns/test/homo/subtask2-homographic-test.gold"
puns,punWord = simpleReadXml(pathData,pathLabel)

In [88]:
wordMap = {} # 找出所有的单词 word => list
for pun in puns:
    for word in pun: # 找出所有的单词
        if word not in wordMap.keys():
            wordMap[word] = []
        else:
            continue
print(len(wordMap)) # 有4567 个词

4567


In [155]:
words = list(wordMap.keys()) # 转为list
from nltk.corpus import wordnet as wn
# step1.获取所有单词
for word in words:
    senses = wn.synsets(word)
    gross = []
    for _ in senses:        
        gross.append(_.definition())
    print(gross)
    
    # step2.处理定义
    if len(gross)<1:
        val = word + " None\n"
        with open(path,'a') as f:
            f.write(val)
    else:    
        inputs = tokenizer(gross,
                      padding='max_length',
                      truncation=True,
                       max_length=40,
                       return_tensors='pt')
        out = model(**inputs)
        last_layer,other = out
        cls_emb = last_layer[:,0,:] # 只取 CLS 上的值

        # step3.将结果写入到文件中
        cls_emb = cls_emb.tolist()
        with open(path,'a') as f: # 写入单词
            f.write(word+" ")
        for emb in cls_emb:    
            res = ""
            for data in emb:
                #print(data)
                res += (str(data)) 
                res += " "
            res.strip(" ")
            res+="\n"
            with open(path,'a') as f:
                f.write(res)

[]
['prevent from being seen or discovered', 'be or go into hiding; keep out of sight, as for protection and safety', 'cover as if with a shroud', 'make undecipherable or imperceptible by obscuring or concealing']
[]
[]
['a professional killer who uses a gun', 'a person who shoots a gun (as regards their ability)']
['a unit of length equal to one twelfth of a foot', 'a rare soft silvery metallic element; occurs in small quantities in sphalerite', 'a state in midwestern United States', 'holding office', 'directed or bound inward', 'currently fashionable', 'to or toward the inside of']
['a metric unit of length equal to one ten billionth of a meter (or 0.0001 micron); used to specify wavelengths of electromagnetic radiation', 'any of several fat-soluble vitamins essential for normal vision; prevents night blindness or inflammation or dryness of the eyes', 'one of the four nucleotides used in building DNA; all four nucleotides have a common phosphate group and a sugar (ribose)', '(biochem

['the state of being in effect or being operative', 'a business especially one run on a large scale', 'a planned activity involving many people performing various actions', '(computer science) data processing in which the result is completely specified by a rule (especially the processing that results from a single instruction)', 'activity by a military or naval force (as a maneuver or campaign)', 'a medical procedure involving an incision with instruments; performed to repair damage or arrest disease in a living body', 'a process or series of acts especially of a practical or mechanical nature involved in a particular form of work', 'process or manner of functioning or operating', '(psychology) the performance of some composite cognitive activity; an operation that affects mental contents', '(mathematics) calculation by mathematical methods', 'the activity of operating something (a machine or business etc.)']
[]
['a metric unit of length equal to one ten billionth of a meter (or 0.000

['sloping land (especially the slope beside a body of water)', 'a financial institution that accepts deposits and channels the money into lending activities', 'a long ridge or pile', 'an arrangement of similar objects in a row or in tiers', 'a supply or stock held in reserve for future use (especially in emergencies)', 'the funds held by a gambling house or the dealer in some gambling games', 'a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force', 'a container (usually with a slot in the top) for keeping money at home', 'a building in which the business of banking transacted', 'a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)', 'tip laterally', 'enclose with a bank', 'do business with a bank or keep an account at a bank', 'act as the banker in a game or in gambling', 'be in the banking business', 'put into a bank account', 'cover with ashes so to control the rate o

['in operation or operational', '(of events) planned or scheduled', 'with a forward motion', 'indicates continuity or persistence or concentration', 'in a state required for something to function or be effective']
[]
['on one occasion', 'as soon as', 'at a previous time']
['water frozen in the solid state', 'the frozen part of a body of water', 'diamonds', 'a flavored sugar topping used to coat and decorate cakes', 'a frozen dessert with fruit flavoring (especially one containing no milk)', 'an amphetamine derivative (trade name Methedrine) used in the form of a crystalline hydrochloride; used as a stimulant to the nervous system and as an appetite suppressant', 'a heat engine in which combustion occurs inside the engine rather than in a separate furnace; heat expands a gas that either moves a piston or turns a gas turbine', 'a rink with a floor of ice for ice hockey or ice skating', 'decorate with frosting', 'cause to become ice or icy', 'put ice on or put on ice']
['the best people o

[]
['move forward, also in the metaphorical sense', 'bring forward for consideration or acceptance', 'increase or raise', 'contribute to the progress or growth of', 'cause to move forward', 'obtain advantages, such as points, etc.', 'develop in a positive way', 'develop further', 'give a promotion to or assign to a higher position', 'pay in advance', 'move forward', 'rise in rate or price', 'farther along in physical or mental development', 'comparatively late in a course of development', 'ahead of the times', 'at a higher level in training or knowledge or skill', 'ahead in development; complex or intricate', 'far along in time', '(of societies) highly developed especially in technology or industry', 'situated ahead or going before']
['the pure mathematics of points and lines and curves and surfaces']
['a collection of things sharing a common attribute', 'a body of students who are taught together', 'people having the same social, economic, or educational status', 'education imparted i

...

['how something is done or how it happens', 'characteristic or habitual practice', 'the latest and most admired style in clothes and cosmetics and behavior', 'consumer goods (especially clothing) in the current mode', 'make out of components (often in an improvising manner)']
['cause to change; make different; cause a transformation', "undergo a change; become different in essence; losing one's or its original nature", "become different in some particular way, without permanently losing one's or its former characteristics or essence", 'lay aside, abandon, or leave for another', 'change clothes; put on different clothes', 'exchange or replace with another, usually of the same kind or category', 'give to, and receive from, one another', 'change from one vehicle or transportation line to another', 'become deeper in tone', 'remove or replace the coverings of', 'made or become different in nature or form', 'made or become different in some respect', 'changed in constitution or structure or 

['a disposition to be humble; a lack of false pride', 'a humble feeling']
['reach a destination; arrive by movement or progress', 'succeed in a big way; get to the top']
['the coldest season of the year; in the northern hemisphere it extends from the winter solstice to the vernal equinox', 'spend the winter']
['a physician who specializes in psychiatry']
['(Islam) an invisible spirit mentioned in the Koran and believed by Muslims to inhabit the earth and influence mankind by appearing in the form of humans or animals']
['any strong feeling']
['store (liquids or gases) in bottles', 'put into bottles']
['fighting with the fists', 'the enclosure of something in a package or box', 'put into a box', 'hit with the fist', 'engage in a boxing match']
['a characteristic sound', 'a toroidal shape', 'a rigid circular band of metal or wood or other material used for holding or fastening or hanging or pulling', '(chemistry) a chain of atoms in a molecule that forms a closed loop', 'an association o

['discover or come upon accidentally, suddenly, or unexpectedly; catch somebody doing something or in a certain state', 'perceive with the senses quickly, suddenly, or momentarily', 'reach with a blow or hit in a particular spot', 'take hold of so as to seize or restrain or stop the motion of', 'succeed in catching or seizing, especially after a chase', 'to hook or entangle', 'attract and fix', 'capture as if by hunting, snaring, or trapping', 'reach in time', 'get or regain something necessary, usually quickly or briefly', 'catch up with and possibly overtake', 'be struck or affected by', 'check oneself during an action', 'hear, usually without the knowledge of the speakers', 'see or watch', 'cause to become accidentally or suddenly caught, ensnared, or entangled', 'detect a blunder or misstep', 'grasp with the mind or develop an understanding of', 'contract', 'start burning', 'perceive by hearing', 'suffer from the receipt of', 'attract; cause to be enamored', 'apprehend and reproduc

['a weapon that discharges a missile at high velocity (especially from a metal tube or barrel)', 'large but transportable armament', 'a person who shoots a gun (as regards their ability)', 'a professional killer who uses a gun', 'a hand-operated pump that resembles a revolver; forces grease into parts of a machine', 'a pedal that controls the throttle valve', 'the discharge of a firearm as signal or as a salute in military ceremonies', 'shoot with a gun']
['a projectile that is fired from a gun', 'a high-speed passenger train', '(baseball) a pitch thrown with maximum velocity']
["a specialist who studies processes in the earth's atmosphere that cause weather conditions"]
['the act of leaving secretly or without explanation', 'get lost, as without warning or explanation', 'become invisible or unnoticeable', 'cease to exist', 'become less intense and fade away gradually']
['any of several large humped bovids having shaggy manes and large heads and short horns']
["to a complete degree or 

KeyboardInterrupt: 

In [147]:
"""检测生成的数据
"""
wordEmb={}# str => list
with open(path,'r') as f:
    line = f.readline()    
    while(line): # 只要有内容，就要尝试                
        print(line)
        line = line.split() # 先按照空格分割
        if line[-1] == "None": # 以None结尾
            wordEmb[line[0]] = None            
        elif line[0].isalpha(): # 如果是字符。是一个新的开始            
            wordEmb[res1] = emb
            emb = [] # 装下当下单词所有的emb
            res1 = line[0] # 得到单词
            del(line[0])
            line = [float(_) for _ in line] # 全部转为float 型
            emb.append(line)            
        else:            
            line = [float(_) for _ in line] # 全部转为float 型
            print(line)
            emb.append(line)
        line = f.readline()
print(wordEmb)

CLS None

['CLS', 'None']


AttributeError: 'list' object has no attribute 'split'

In [148]:
li= [1,2,3,4]
a = li
print(a)

[1, 2, 3, 4]


In [149]:
del(li[0])
print(a)

[2, 3, 4]


In [150]:
print(id(li))

139952127937024


In [151]:
li = []
print(id(li))

139952127090048


In [154]:
text = ['one of the twelve divisions of the calendar year', 'a time unit of approximately 30 days']
inps = tokenizer(text,
                      padding='max_length',
                      truncation=True,
                       max_length=40,
                       return_tensors='pt')
ou = model(**inps)
a,b = ou
print(a)

tensor([[[ 4.4885e-01, -3.6890e-01, -2.0316e-01, -7.4981e-02, -2.2937e-01,
           2.0985e-02,  4.5975e-01, -3.4644e-02,  3.4081e-01, -9.4962e-01,
          -3.0050e-01,  3.2282e-02,  2.3876e-01, -2.4972e-01,  2.9231e-02,
          -2.6954e-02,  3.9218e-01,  8.2633e-02, -9.8558e-02, -1.4816e-01,
           2.6315e-01, -1.1277e-01,  6.6885e-01, -6.7628e-02,  1.9755e-01,
          -4.6039e-02,  9.7735e-02, -2.9179e-01,  2.7934e-01,  3.4619e-01,
          -3.6468e-02,  7.7602e-02,  4.2290e-01,  1.3953e-01,  3.9131e-01,
           3.4973e-01,  1.1111e-01, -2.4791e-01,  1.8006e-01,  2.7131e-01,
          -7.6173e-01,  4.0372e-02,  4.6110e-01,  8.9571e-02,  3.6053e-01,
          -2.8257e-01,  1.0637e-01,  4.7342e-03, -1.7545e-01,  1.1994e-02,
           4.7926e-02,  8.1373e-02,  5.7833e-01,  3.7764e-02,  3.3945e-01,
          -1.0871e-01, -2.1502e-01,  1.3865e-02, -1.5184e-01,  2.7772e-01,
           5.6287e-01,  6.6608e-02,  4.5661e-01, -1.1425e-01, -7.9546e-02,
           1.4770e-01,  6